## 2번 문제

- 개요
 - 주어진 데이터는 음성을 텍스트로 변환한 후 감성을 라벨링하였습니다. 
 - 해당 데이터를 바탕으로, 7가지의 감성을 분류하는 모형을 만들어보세요.

- 데이터
 - data.csv
 - 위 데이터는 aihub.or.kr에서 다운로드하였습니다.


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 5.8 MB/s 
     |████████████████████████████████| 465 kB 59.8 MB/s 


In [ ]:
import pandas as pd

data = pd.read_csv('data.csv', encoding='cp949', header=None, skiprows=1)
data.head(3)

0                                          1   \
0  5ed10dbc2880d70f286121c3  개를 예쁘다고 사놓고 끝까지 키우지도 않고 버리는 사람들이 엄청 많아졌대.   
1  5ecb60ef9aa8ea0eec53edb1                    지금도 그대로 있어. 치우는 사람이 없어.   
2  5f052858b140144dfcfef768              맞아. 무기력증인 것 같아. 한동안 정말 바빴었거든.   

        2        3    4        5    6        7    8        9    10       11  \
0  disgust    Angry  2.0    Angry  2.0    Angry  2.0    Angry  2.0    Angry   
1  disgust  Neutral  0.0  Disgust  2.0  Sadness  2.0  Disgust  2.0  Disgust   
2      sad  Sadness  2.0  Sadness  1.0  Sadness  2.0  Sadness  2.0  Sadness   

    12    13      14  
0  2.0  33.0  female  
1  1.0  48.0  female  
2  2.0  48.0  female

In [ ]:
train_df = pd.DataFrame( data.loc[:, 1:2])

In [ ]:
train_df.columns = ["text", "target"]

In [ ]:
train_df.head()

text   target
0    개를 예쁘다고 사놓고 끝까지 키우지도 않고 버리는 사람들이 엄청 많아졌대.  disgust
1                      지금도 그대로 있어. 치우는 사람이 없어.  disgust
2                맞아. 무기력증인 것 같아. 한동안 정말 바빴었거든.      sad
3  오늘이 발표날인데 연락이 없더라고. 그래서 알아봤더니 명단에 내 이름이 없대.      sad
4     그치. 개 키우는 사람이 늘어나니까 그만큼 버리는 사람도 늘어나는 거야!  disgust

In [ ]:
train_df['target'].value_counts( )

sad        7312
anger      4858
fear       1768
disgust    1408
neutral       6
sa            1
Name: target, dtype: int64

In [ ]:
import numpy as np
import re
train_df = train_df.fillna(' ')
# 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['text'] = train_df['text'].apply( lambda x : re.sub(r"\d+", " ", x) )

In [ ]:
from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.nouns(text)
    return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['text'])
tfidf_matrix_train = tfidf_vect.transform(train_df['text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( tfidf_matrix_train, train_df.target)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit( X_train, y_train)

RandomForestClassifier()

In [ ]:
print(rf.score( X_train, y_train))
rf.score( X_test, y_test)

0.9639569220079903


0.8773117999479031

In [ ]:
from sklearn.metrics import classification_report
print( classification_report(rf.predict(X_test), y_test))

              precision    recall  f1-score   support

       anger       0.86      0.88      0.87      1210
     disgust       0.77      0.96      0.85       280
        fear       0.75      0.87      0.81       356
     neutral       0.00      0.00      0.00         0
         sad       0.94      0.86      0.90      1993

    accuracy                           0.88      3839
   macro avg       0.66      0.72      0.69      3839
weighted avg       0.88      0.88      0.88      3839



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Parameter 최적화를 위해 GridSearchCV 를 이용. 
params = { 'n_estimators': [50,75,100,125 ] }
grid_cv = GridSearchCV(rf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(X_train, y_train )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


{'n_estimators': 125} 0.8666


In [ ]:
# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
test_df = pd.DataFrame( {'document':["날씨가 좋아요", "개를 키우자", "소름돋는데"]})
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
preds = rf.predict(tfidf_matrix_test)

print(preds)

['sad' 'disgust' 'sad']


In [ ]:
#다른 모형: import XGboost

from sklearn import model_selection
from xgboost import XGBClassifier

#Defining XGB Classification model
clf = XGBClassifier()

param_grid = {
    "n_estimators":[200,300],
    "max_depth":[1,3],
    "learning_rate":[.01,.1]    
}

model = model_selection.GridSearchCV(
    estimator = clf,
    param_grid = param_grid,
    scoring = 'accuracy',
    verbose = 10,
    n_jobs = 1,
    cv = 3    
)

#fitting values to the gridsearchcv model
model.fit(X_train,y_train)
print(model.best_params_)
print(model.best_estimator_)
print(model.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3; 1/8] START learning_rate=0.01, max_depth=1, n_estimators=200...........


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV 1/3; 1/8] END learning_rate=0.01, max_depth=1, n_estimators=200;, score=0.595 total time=   2.7s
[CV 2/3; 1/8] START learning_rate=0.01, max_depth=1, n_estimators=200...........
[CV 2/3; 1/8] END learning_rate=0.01, max_depth=1, n_estimators=200;, score=0.628 total time=   2.6s
[CV 3/3; 1/8] START learning_rate=0.01, max_depth=1, n_estimators=200...........
[CV 3/3; 1/8] END learning_rate=0.01, max_depth=1, n_estimators=200;, score=0.579 total time=   2.2s
[CV 1/3; 2/8] START learning_rate=0.01, max_depth=1, n_estimators=300...........
[CV 1/3; 2/8] END learning_rate=0.01, max_depth=1, n_estimators=300;, score=0.663 total time=   5.1s
[CV 2/3; 2/8] START learning_rate=0.01, max_depth=1, n_estimators=300...........
[CV 2/3; 2/8] END learning_rate=0.01, max_depth=1, n_estimators=300;, score=0.656 total time=   3.8s
[CV 3/3; 2/8] START learning_rate=0.01, max_depth=1, n_estimators=300...........
[CV 3/3; 2/8] END learning_rate=0.01, max_depth=1, n_estimators=300;, score=0.655 total ti